In [ ]:
%matplotlib widget

import pandas as pd
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
from datetime import date, timedelta

In [ ]:
def read(file):
    _df = pd.read_csv(file, date_format='%d.%m.%Y, %H:%M:%S', index_col ="Timestamp")
    _df['energy'] = _df['Energy in Wm (MRAM)'] / 60 / 1000
    return _df['energy']

folder_path = './data/pv'
files = filter(lambda f: not f.startswith('.'), os.listdir(folder_path))
frames = map(read, map(lambda f: folder_path + '/' + f, files))
df = pd.concat(frames)

In [ ]:
today = date.today()

start_date = today
end_date = today
resample = 0

presets = [
    (0, 'Today', today, today + timedelta(days=1), 2),
    (1, 'Yesterday', today - timedelta(days=1), today, 2),
    (2, 'Last Week', today - timedelta(days=7), today, 7),
    (3, 'Last Week (incl. Today)', today - timedelta(days=7), today + timedelta(days=1), 7),
    (4, 'Last 30 Days', today - timedelta(days=30), today, 7),
    (4, 'Last 90 Days', today - timedelta(days=90), today, 7),
]
presets_picker = widgets.Dropdown(
    description='Presets:',
    value=3,
    options=map(lambda d: (d[1], d[0]), presets),
)
from_picker = widgets.DatePicker(
    description='From',
    value=start_date,
    disabled=False
)
to_picker = widgets.DatePicker(
    description='To',
    value=end_date,
    disabled=False
)
resample_options = [
    (0, '5m', '5T'),
    (1, '10m', '10T'),
    (2, '15m', '15T'),
    (3, '30m', '30T'),
    (4, '1h', '1H'),
    (5, '6h', '6H'),
    (6, '12h', '12H'),
    (7, '1D', '1D'),
    (8, '1W', '1W')
]
resample_picker = widgets.Dropdown(
    description='Resample',
    value=resample,
    options=map(lambda d: (d[1], d[0]), resample_options)
)
output_detail = widgets.HTML()
output_plot = widgets.Output()


def filter_df(df, start_date, end_date, resample):
    _df = df.loc[(df.index >= pd.Timestamp(str(start_date))) & (df.index <= pd.Timestamp(str(end_date)))]
    return _df.resample(resample_options[resample][2]).sum()

def get_details_table(data):
    rows = map(
        lambda d: f"""
        <tr>
            <th><b>{d[0]}</b></th>
            <td>{d[1]:.4f} kWh</td>
        </tr>
        """, 
        data
    )
    return f"<table>{''.join(rows)}</table>"

def print_df_details(df):
    details = [
        ('Sum', df.sum()),
        ('Median', df.median()),
        ('Mean', df.mean()),
        ('Standard Deviation', df.std()),
        ('Minimum', df.min()),
        ('Maximum', df.max()),
    ]
    output_detail.value = get_details_table(details)

def draw_df(df):
    ax.clear()
    ax.autoscale()
    ax.set_title('PV Energy')
    ax.set_xlabel('Timestamp')
    ax.set_ylabel('Energy [kWh]')
    ax.grid(True)
    ax.plot(df, color='C0', marker='o')
    ax.set_ylim(bottom=0.0, auto=True)

def update():
    _df = filter_df(df, start_date, end_date, resample)
    print_df_details(_df)
    draw_df(_df)


def update_presets(value):
    global start_date
    global end_date
    global resample
    
    selected = presets[value]

    from_picker.value = selected[2]
    start_date = selected[2]

    to_picker.value = selected[3]
    end_date = selected[3]

    resample_picker.value = selected[4]
    resample = selected[4]
presets_picker.observe(lambda v: update_presets(v.new), 'value')

def update_from(value):
    global start_date
    start_date = value
    update()
from_picker.observe(lambda v: update_from(v.new), 'value')

def update_to(value):
    global end_date
    end_date = value
    update()
to_picker.observe(lambda v: update_to(v.new), 'value')

def update_resample(value):
    global resample
    resample = value
    update()
resample_picker.observe(lambda v: update_resample(v.new), 'value')

with output_plot:
    fig, ax = plt.subplots(constrained_layout=True, figsize=(16, 8))
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False

update_presets(3)

controls = widgets.VBox([presets_picker, widgets.Label(), from_picker, to_picker, resample_picker])
header = widgets.HBox([controls, output_detail])
header.layout = widgets.Layout(
        width='70%',
        display='flex',
        justify_content='space-around',
        align_items='center'
)
widgets.VBox([output_plot, header])